In [1]:
import numpy as np
import pandas as pd 
from bs4 import BeautifulSoup
import urllib.request
import os
from google_patent_scraper import scraper_class
import json
import timeit
import networkx as nx
import matplotlib.pyplot as plt

In C:\Users\carlo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\carlo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\carlo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\carlo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\carlo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [203]:
G = nx.read_graphml("7topics.graphml") #when reading you should change the attibute names
[G.nodes[node] for node in G.nodes]

[{'class': 'G06', 'topic': '4', 'class_topic': 'G06_4'},
 {'class': 'H04', 'topic': '4', 'class_topic': 'H04_4'},
 {'class': 'H04', 'topic': '4', 'class_topic': 'H04_4'},
 {'class': 'G06', 'topic': '4', 'class_topic': 'G06_4'},
 {'class': 'G06', 'topic': '2', 'class_topic': 'G06_2'},
 {'class': 'H04', 'topic': '4', 'class_topic': 'H04_4'},
 {'class': 'G06', 'topic': '6', 'class_topic': 'G06_6'},
 {'class': 'G06', 'topic': '7', 'class_topic': 'G06_7'},
 {'class': 'G06', 'topic': '1', 'class_topic': 'G06_1'},
 {'class': 'H04', 'topic': '0', 'class_topic': 'H04_0'},
 {'class': 'H04', 'topic': '1', 'class_topic': 'H04_1'},
 {'class': 'H04', 'topic': '0', 'class_topic': 'H04_0'},
 {'class': 'G06', 'topic': '4', 'class_topic': 'G06_4'},
 {'class': 'G06', 'topic': '4', 'class_topic': 'G06_4'},
 {'class': 'G06', 'topic': '6', 'class_topic': 'G06_6'},
 {'class': 'G06', 'topic': '6', 'class_topic': 'G06_6'},
 {'class': 'G06', 'topic': '6', 'class_topic': 'G06_6'},
 {'class': 'G06', 'topic': '2',

In [20]:
components =  [c for c in sorted(nx.connected_components(G), key=len, reverse=True)]
for c in components: print(len(c),nx.average_clustering(G,c))

8104 0.11368707831513386
120 0.563941231886285
68 0.020647653000594176
45 0.0
22 0.0
18 0.0
17 0.08972073677956031
13 0.0
13 0.0
12 0.0
11 0.0
10 0.0
9 0.0
8 0.0
8 0.175
8 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.0
7 0.3
7 0.0
6 0.0
6 0.0
6 0.0
6 0.0
6 0.0
6 0.0
6 0.0
6 0.0
6 0.0
6 0.0
6 0.0
5 0.0
5 0.0
5 0.0
5 0.3333333333333333
5 0.0
5 0.0
5 0.0
5 0.0
5 0.0
5 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.5833333333333334
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
4 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
3 0.0
2 0.0
2 

In [24]:
np.average([len(c) for c in components])

24.359897172236504

In [25]:
g = G.copy()
len(g.nodes)

9476

In [ ]:
def remove_components(g, minsize=25):
    components =  [c for c in sorted(nx.connected_components(g), key=len, reverse=True)]
    G=g.subgraph(components[0]).copy()
    return components
    
    

In [33]:

def random_centroids(G,k):
    import random
    return random.sample(G.nodes,k)

In [76]:
def compute_clusters(G,centroids):
    '''
    assign each node to its closest centroid, using dijkstra algorithm to compute shortest path
    '''
    clusters = [[] for cluster in range(len(centroids))]
    diameter = 100
    for node in G.nodes:
        shortest_distance = diameter
        clust = None
        for i,centroid in enumerate(centroids):
            distance = nx.shortest_path_length(G, source=node, target=centroid)
            if  distance< shortest_distance:
                clust = i
                shortest_distance = distance
        #nx.set_node_attributes(G, {node: clust}, name='cluster') 
        clusters[clust].append(node)
    return clusters
        
            

In [64]:
def compute_centroids(G,clusters):
    ''' for each cluster, from the obtained subgraph
        the centroid is selected as the node with greatest clustering coefficient
    '''
    centroids = list()
    for cluster in clusters:
        C = G.subgraph(c).copy()
        centroids.append(cluster[np.argmax(nx.clustering(C))])
    return centroids

In [81]:
def knodes(G,k=5,maxiter=100):
    
    #select k random initial centroids and
    #compute clusters
    centroids = random.sample(G.nodes,k)
    clusters = compute_clusters(G,centroids)
    print('started', end=' ')
    for it in range(maxiter):
        
        new_centroids = compute_centroids(G,clusters)
        clusters = compute_clusters(G,centroids)
       
        if new_centroids == centroids: break
        centroids = new_centroids
        print(it,centroids, [len(c) for c in clusters])
    
    return clusters
    
    

In [82]:
clusters = knodes(G)

started 0 ['US8370262B2', 'US9838407B1', 'US9917860B2', 'US20080174100A1', 'US9848009B2'] [4491, 1328, 1765, 297, 223]


In [83]:
nx.set_node_attributes(g, 0, name='cluster')
for i,c in enumerate(clusters):
    for node in c:
        nx.set_node_attributes(g, {node: i+1}, name='cluster') 

In [84]:
nx.write_graphml(g, "../data/clusters.graphml")